In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv(
    "../data/Hearthstone - most watched Twitch channels - SullyGnome1-100.csv"
)

df2 = pd.read_csv(
    "../data/Hearthstone - most watched Twitch channels - SullyGnome101-200.csv"
)

In [3]:
df = pd.concat([df1, df2], ignore_index=True)

In [4]:
df = (
    df.rename(columns={"Unnamed: 0": "Avatar", "Unnamed: 1": "Rank"})
    .drop(["Unnamed: 13"], axis=1)
    .rename(columns=str.lower)
    .rename(columns=lambda x: "_".join(x.split()))
    
)

In [38]:
twitter_urls = []

In [ ]:
for channel in df['channel']:
    print(f"https://twitch.tv/{channel}/about")
    twitter_url = input("Twitter URL?")
    twitter_urls.append(t)

In [42]:
df['twitter_link'] = twitter_urls

In [45]:
df.to_csv("../data/hearthstone-twitch-data.csv")